In [1]:
#filedrive="/content/drive/My Drive/0Satellite_Images_Unsupervised/MNIST_PyTorch"
#from google.colab import drive
#drive.mount('/content/drive')
filedrive="data/0Satellite_Images_Unsupervised/MNIST_PyTorch"

import torch, torchvision
from torch import nn, optim

In [2]:
class Classifier(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.layer1 = nn.Linear(784, 20)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(20, 10)
#        self.layer = nn.Linear(10, 10)

    def forward(self, x):
        x=self.layer1(x)
        x=self.act1(x)
        return self.layer2(x)
#  use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load model to the specified device, either gpu or cpu
model = Classifier().to(device)

# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# mean-squared error loss
criterion = nn.CrossEntropyLoss()

In [3]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

train_dataset = torchvision.datasets.MNIST(
    root=filedrive, train=True, transform=transform, download=True)

test_dataset = torchvision.datasets.MNIST(
    root=filedrive, train=False, transform=transform, download=True)

In [4]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [8]:
epochs = 5

from tqdm import tqdm
    
pbar = tqdm(range(epochs))
for epoch in pbar:
    loss = 0
    for batch_features, batch_labels in train_loader:
        # reshape mini-batch data to [N, 784] matrix
        # load it to the active device
        batch_features = batch_features.view(-1, 784).to(device)

        # reset the gradients back to zero
        # PyTorch accumulates gradients on subsequent backward passes
        optimizer.zero_grad()

        # compute reconstructions
        outputs = model(batch_features)
        #outputs = outputs.view(-1).to(device)

        # compute training reconstruction loss

        train_loss = criterion(outputs, batch_labels.to(device))

        # compute accumulated gradients
        train_loss.backward()

        # perform parameter update based on current gradients
        optimizer.step()

        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()

    # compute the epoch training loss
    loss = loss / len(train_loader)

    # display the epoch training loss
    pbar.set_description("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))
    print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))

epoch : 1/5, loss = 0.213790:  20%|██        | 1/5 [00:11<00:44, 11.02s/it]

epoch : 1/5, loss = 0.213790


epoch : 2/5, loss = 0.198452:  40%|████      | 2/5 [00:22<00:33, 11.04s/it]

epoch : 2/5, loss = 0.198452


epoch : 3/5, loss = 0.185208:  60%|██████    | 3/5 [00:33<00:21, 11.00s/it]

epoch : 3/5, loss = 0.185208


epoch : 4/5, loss = 0.174383:  80%|████████  | 4/5 [00:44<00:11, 11.13s/it]

epoch : 4/5, loss = 0.174383


epoch : 5/5, loss = 0.164760: 100%|██████████| 5/5 [00:55<00:00, 11.07s/it]

epoch : 5/5, loss = 0.164760


In [7]:
import matplotlib.pyplot as plt
import numpy as np
total = 0
correct = 0
print(test_loader.dataset)
for batch_features, labels in test_loader:
    batch_features = batch_features.view(-1, 784).to(device)  
    outputs = model(batch_features)
    #print(outputs.shape)

    #print('IMAGE')

    #plt.imshow(batch_features[0].cpu().detach().view(28, 28).numpy())
    #plt.show()     

    #print(np.argmax(outputs[0].cpu().detach().numpy()))   

    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Dataset MNIST
    Number of datapoints: 10000
    Root location: data/0Satellite_Images_Unsupervised/MNIST_PyTorch
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
           )
Accuracy of the network on the 10000 test images: 93 %
